# Build some workbooks with requisite data for modelling industry

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from openpyxl import Workbook
import xlsxwriter
import pandas.io.formats.excel
import glob
from pandas import ExcelWriter

In [96]:
# Load data frames

EGEDA_years = pd.read_csv('../data/input_data/EGEDA_to2017.csv')
historical_production = pd.read_csv('../data/input_data/historical_production.csv')
GDP = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'GDP')
GDP_growth = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'GDP_growth')
population = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'Population')
GDP_per_capita = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'GDP_per_capita')
GDP_per_capita_percent = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'GDP_per_capita_%')
sectoral_GDP = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'Sectoral_GDP')
sectoral_GDP_percent = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'Sectoral_GDP_%')

In [109]:
# Some things required to build the workbook

# Economy vector
economies = historical_production['economy'].unique()

# Make space for charts
chart_height = 18 # number of excel rows

# Series

series = ['GDP', 'GDP growth', 'Population', 'GDP per capita', 'GDP per capita growth']
sector = ['agriculture_gdp_ppp_2017_usd', 'industry_gdp_ppp_2017_usd', 'manufacturing_gdp_ppp_2017_usd', 'services_gdp_ppp_2017_usd', 'agriculture_gdp_share',
          'industry_gdp_share', 'manufacturing_gdp_share', 'services_gdp_share']
chemicals = ['ammonia_production', 'ethylene_production', 'propylene_production', 'benzene_production', 'toluene_production', 'xylene_production']
pulp_paper = ['pulp_mechanical_production', 'pulp_semi_chemical_production', 'pulp_chemical_production', 'pulp_recovered_production', 'paper_news_production', 
              'paper_writing_production', 'paper_paperboard_production', 'paper_recovered_production']

# Colours

colours = pd.read_excel('../data/colour_template_7th.xlsx')
colours_hex = colours['hex']

In [116]:
##################################### Build initial look workbook #####################################

# Define directory where to save the workbook
script_dir = '../results/'
results_dir = os.path.join(script_dir, 'initial_look/')
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

# Create a pandas ExcelWriter workbook using xlsxwriter as the engine and save it to the directory created above

writer = pd.ExcelWriter(results_dir + 'initial_look.xlsx', engine = 'xlsxwriter')
pandas.io.formats.excel.ExcelFormatter.header_style = None

# Make workbooks with requisite data

for economy in economies:
    # Empty macro dataframe and empty sectoral df
    macro_df = pd.DataFrame()
    sectoral_df = pd.DataFrame()

    # The requisite macro data to populate macro dataframe
    GDP_build = GDP[GDP['Economy'] == economy].reset_index(drop = True)
    GDP_build['Series'] = 'GDP'
    GDP_growth_build = GDP_growth[GDP_growth['Economy'] == economy].reset_index(drop = True)
    GDP_growth_build['Series'] = 'GDP growth'
    pop_build = population[population['Economy'] == economy].reset_index(drop = True)
    pop_build['Series'] = 'Population' 
    GDPpercapita_build = GDP_per_capita[GDP_per_capita['Economy'] == economy].reset_index(drop = True)
    GDPpercapita_build['Series'] = 'GDP per capita'
    GDPpercapita_pc_build = GDP_per_capita_percent[GDP_per_capita_percent['Economy'] == economy].reset_index(drop = True)
    GDPpercapita_pc_build['Series'] = 'GDP per capita growth'

    macro_df = macro_df.append([GDP_build, GDP_growth_build, pop_build, GDPpercapita_build, GDPpercapita_pc_build]).reset_index(drop = True)[['Economy', 'Series', 'Unit'] + list(range(1990, 2051, 1))]

    macro_rows = macro_df.shape[0]
    macro_cols = macro_df.shape[1]
    
    # Sectoral df
    sectGDP_build = sectoral_GDP[sectoral_GDP['Economy'] == economy].reset_index(drop = True)
    sectGDPpc_build = sectoral_GDP_percent[sectoral_GDP_percent['Economy'] == economy].reset_index(drop = True)

    sectoral_df = sectoral_df.append([sectGDP_build, sectGDPpc_build]).reset_index(drop = True)

    sectoral_rows = sectoral_df.shape[0]
    sectoral_cols = sectoral_df.shape[1]

    # Now create the historical production dataframes

    steel_df1 = historical_production[(historical_production['item'] == 'steel_production') &
                                      (historical_production['economy'] == economy)].reset_index(drop = True)

    cement_df1 = historical_production[(historical_production['item'] == 'cement_production') &
                                       (historical_production['economy'] == economy)].reset_index(drop = True)

    aluminium_df1 = historical_production[(historical_production['item'] == 'aluminium_production') &
                                           (historical_production['economy'] == economy)].reset_index(drop = True)

    chemicals_df1 = historical_production[(historical_production['item'].isin(['ammonia_production', 'ethylene_production', 'propylene_production', 'benzene_production', 
                                                                               'toluene_production', 'xylene_production'])) &
                                          (historical_production['economy'] == economy)].reset_index(drop = True)

    chemical_rows = chemicals_df1.shape[0]
    chemical_cols = chemicals_df1.shape[1]

    pulppaper_df1 = historical_production[(historical_production['item'].isin(['pulp_mechanical_production', 'pulp_semi_chemical_production', 'pulp_chemical_production',
                                                                               'pulp_recovered_production', 'paper_news_production', 'paper_writing_production', 'paper_paperboard_production', 
                                                                               'paper_recovered_production'])) &
                                          (historical_production['economy'] == economy)].reset_index(drop = True)

    pulppaper_rows = pulppaper_df1.shape[0]
    pulppaper_cols = pulppaper_df1.shape[1]

    mining_df1 = historical_production[(historical_production['item'] == 'mining_gva') &
                                       (historical_production['economy'] == economy)].reset_index(drop = True)

    fb_df1 = historical_production[(historical_production['item'] == 'food_beverage_tobacco_gva') &
                                   (historical_production['economy'] == economy)].reset_index(drop = True)

    textiles_df1 = historical_production[(historical_production['item'] == 'textiles_gva') &
                                         (historical_production['economy'] == economy)].reset_index(drop = True)

    wood_df1 = historical_production[(historical_production['item'] == 'wood_gva') &
                                     (historical_production['economy'] == economy)].reset_index(drop = True)

    machinery_df1 = historical_production[(historical_production['item'] == 'machinery_gva') &
                                          (historical_production['economy'] == economy)].reset_index(drop = True)

    vehicles_df1 = historical_production[(historical_production['item'] == 'vehicles_gva') &
                                         (historical_production['economy'] == economy)].reset_index(drop = True)

    oth_trans_df1 = historical_production[(historical_production['item'] == 'other_transport_gva') &
                                          (historical_production['economy'] == economy)].reset_index(drop = True)

    construction_df1 = historical_production[(historical_production['item'] == 'construction_gva') &
                                             (historical_production['economy'] == economy)].reset_index(drop = True)                                           
                                                                                  

    # place the dataframes within the initial look workbook
    macro_df.to_excel(writer, sheet_name = economy, index = False, startrow = chart_height)
    sectoral_df.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 2) + macro_rows + 1)
    steel_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 3) + macro_rows + sectoral_rows + 2)
    cement_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 4) + macro_rows + sectoral_rows + 4)
    aluminium_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 5) + macro_rows + sectoral_rows + 6)
    chemicals_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 6) + macro_rows + sectoral_rows + 8)
    pulppaper_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 7) + macro_rows + sectoral_rows + chemical_rows + 9)
    mining_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 8) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 10)
    fb_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 9) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 12)
    textiles_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 10) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 14)
    wood_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 11) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 16)
    machinery_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 12) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 18)
    vehicles_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 13) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 20)
    oth_trans_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 14) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 22)
    construction_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 15) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 24) 

    # Access the workbook
    workbook = writer.book
    econ_worksheet = writer.sheets[economy]

    # Comma format and header format
    comma_format = workbook.add_format({'num_format': '#,##0.00'})
    header_format = workbook.add_format({'font_name': 'Calibri', 'font_size': 11, 'bold': True})
    cell_format1 = workbook.add_format({'bold': True})

    # Apply formatting
    econ_worksheet.set_column(3, macro_cols + 1, None, comma_format)
    econ_worksheet.set_row(chart_height, None, header_format)
    econ_worksheet.set_row((chart_height * 2) + macro_rows + 1, None, header_format)
    econ_worksheet.set_row((chart_height * 3) + macro_rows + sectoral_rows + 2, None, header_format)
    econ_worksheet.set_row((chart_height * 4) + macro_rows + sectoral_rows + 4, None, header_format)
    econ_worksheet.set_row((chart_height * 5) + macro_rows + sectoral_rows + 6, None, header_format)
    econ_worksheet.set_row((chart_height * 6) + macro_rows + sectoral_rows + 8, None, header_format)
    econ_worksheet.set_row((chart_height * 7) + macro_rows + sectoral_rows + chemical_rows + 9, None, header_format)
    econ_worksheet.set_row((chart_height * 8) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 10, None, header_format)
    econ_worksheet.set_row((chart_height * 9) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 12, None, header_format)
    econ_worksheet.set_row((chart_height * 10) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 14, None, header_format)
    econ_worksheet.set_row((chart_height * 11) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 16, None, header_format)
    econ_worksheet.set_row((chart_height * 12) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 18, None, header_format)
    econ_worksheet.set_row((chart_height * 13) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 20, None, header_format)
    econ_worksheet.set_row((chart_height * 14) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 22, None, header_format)
    econ_worksheet.set_row((chart_height * 15) + macro_rows + sectoral_rows + chemical_rows + pulppaper_rows + 24, None, header_format)
    econ_worksheet.write(0, 0, economy + ' initial look at industry macro variables and industry production', cell_format1)

    # Create a GDP line chart
    macro1 = workbook.add_chart({'type': 'line'})
    macro1.set_size({
        'width': 500,
        'height': 300
    })
    
    macro1.set_chartarea({
        'border': {'none': True}
    })
    
    macro1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'GDP',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro1.set_title({
        'none': True
    })
        
    for item in series[:1]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro1.add_series({
            'name':       [economy, i + 1 + chart_height, 1],
            'categories': [economy, chart_height, 3, chart_height, macro_cols - 1],
            'values':     [economy, i + 1 + chart_height, 3, i + 1 + chart_height, macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('B3', macro1)

    # Create a GDP growth column chart
    macro2 = workbook.add_chart({'type': 'column'})
    macro2.set_size({
        'width': 500,
        'height': 300
    })
    
    macro2.set_chartarea({
        'border': {'none': True}
    })
    
    macro2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'GDP growth',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro2.set_title({
        'none': True
    })
        
    for item in series[1:2]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro2.add_series({
            'name':       [economy, i + 1 + chart_height, 1],
            'categories': [economy, chart_height, 3, chart_height, macro_cols - 1],
            'values':     [economy, i + 1 + chart_height, 3, i + 1 + chart_height, macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('J3', macro2)

    # Create a population line chart
    macro3 = workbook.add_chart({'type': 'line'})
    macro3.set_size({
        'width': 500,
        'height': 300
    })
    
    macro3.set_chartarea({
        'border': {'none': True}
    })
    
    macro3.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro3.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Population',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro3.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro3.set_title({
        'none': True
    })
        
    for item in series[2:3]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro3.add_series({
            'name':       [economy, i + 1 + chart_height, 1],
            'categories': [economy, chart_height, 3, chart_height, macro_cols - 1],
            'values':     [economy, i + 1 + chart_height, 3, i + 1 + chart_height, macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('R3', macro3)

    # Create a GDP line chart
    macro4 = workbook.add_chart({'type': 'line'})
    macro4.set_size({
        'width': 500,
        'height': 300
    })
    
    macro4.set_chartarea({
        'border': {'none': True}
    })
    
    macro4.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro4.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'GDP per capita',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro4.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro4.set_title({
        'none': True
    })
        
    for item in series[3:4]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro4.add_series({
            'name':       [economy, i + 1 + chart_height, 1],
            'categories': [economy, chart_height, 3, chart_height, macro_cols - 1],
            'values':     [economy, i + 1 + chart_height, 3, i + 1 + chart_height, macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('Z3', macro4)

    # Create a GDP growth column chart
    macro5 = workbook.add_chart({'type': 'column'})
    macro5.set_size({
        'width': 500,
        'height': 300
    })
    
    macro5.set_chartarea({
        'border': {'none': True}
    })
    
    macro5.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro5.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'GDP per capita growth',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro5.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro5.set_title({
        'none': True
    })
        
    for item in series[4:]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro5.add_series({
            'name':       [economy, i + 1 + chart_height, 1],
            'categories': [economy, chart_height, 3, chart_height, macro_cols - 1],
            'values':     [economy, i + 1 + chart_height, 3, i + 1 + chart_height, macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('AH3', macro5)

    ###########################################################################################################################

    # Create a sectoral line chart
    sectoral1 = workbook.add_chart({'type': 'line'})
    sectoral1.set_size({
        'width': 500,
        'height': 300
    })
    
    sectoral1.set_chartarea({
        'border': {'none': True}
    })
    
    sectoral1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    sectoral1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Sectoral GDP',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    sectoral1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10}
        #'none': True
    })
        
    sectoral1.set_title({
        'none': True
    })
        
    for item in sector[4:8]:
        i = sectoral_df[sectoral_df['Sector'] == item].index[0]
        sectoral1.add_series({
            'name':       [economy, i + 7 + (chart_height * 2), 2],
            'categories': [economy, (chart_height * 2) + 6, 3, (chart_height * 2) + 6, sectoral_cols - 1],
            'values':     [economy, i + 7 + (chart_height * 2), 3, i + 7 + (chart_height * 2), sectoral_cols - 1],
            'line':       {'color': colours_hex[i], 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('B27', sectoral1)

    #################################################################################################################################

    # Create a steel line chart
    steel1 = workbook.add_chart({'type': 'line'})
    steel1.set_size({
        'width': 500,
        'height': 300
    })
    
    steel1.set_chartarea({
        'border': {'none': True}
    })
    
    steel1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    steel1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Steel production (thousand tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    steel1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    steel1.set_title({
        'none': True
    })
        
    steel1.add_series({
        'name':       [economy, (chart_height * 3) + 16, 1],
        'categories': [economy, (chart_height * 3) + 15, 4, (chart_height * 3) + 15, steel_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 3) + 16, 4, (chart_height * 3) + 16, steel_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    
    econ_worksheet.insert_chart('B54', steel1)

    #################################################################################################################################

    # Create a cement line chart
    cement1 = workbook.add_chart({'type': 'line'})
    cement1.set_size({
        'width': 500,
        'height': 300
    })
    
    cement1.set_chartarea({
        'border': {'none': True}
    })
    
    cement1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    cement1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Cement production (thousand tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    cement1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    cement1.set_title({
        'none': True
    })
        
    cement1.add_series({
        'name':       [economy, (chart_height * 4) + 18, 1],
        'categories': [economy, (chart_height * 4) + 17, 4, (chart_height * 4) + 17, cement_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 4) + 18, 4, (chart_height * 4) + 18, cement_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    
    econ_worksheet.insert_chart('B74', cement1)

    #################################################################################################################################

    # Create an aluminium line chart
    alum1 = workbook.add_chart({'type': 'line'})
    alum1.set_size({
        'width': 500,
        'height': 300
    })
    
    alum1.set_chartarea({
        'border': {'none': True}
    })
    
    alum1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    alum1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Aluminium production (thousand tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    alum1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    alum1.set_title({
        'none': True
    })
        
    alum1.add_series({
        'name':       [economy, (chart_height * 5) + 20, 1],
        'categories': [economy, (chart_height * 5) + 19, 4, (chart_height * 5) + 19, aluminium_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 5) + 20, 4, (chart_height * 5) + 20, aluminium_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    
    econ_worksheet.insert_chart('B94', alum1)

    ############################################################################################################################

    # Create a chemocals production line chart
    chem1 = workbook.add_chart({'type': 'line'})
    chem1.set_size({
        'width': 700,
        'height': 300
    })
    
    chem1.set_chartarea({
        'border': {'none': True}
    })
    
    chem1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    chem1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Chemicals production (thousand tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    chem1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10}
        #'none': True
    })
        
    chem1.set_title({
        'none': True
    })
        
    for item in chemicals:
        i = chemicals_df1[chemicals_df1['item'] == item].index[0]
        chem1.add_series({
            'name':       [economy, (chart_height * 6) + 22 + i, 1],
            'categories': [economy, (chart_height * 6) + 21, 4, (chart_height * 6) + 21, chemical_cols - 1],
            'values':     [economy, (chart_height * 6) + 22 + i, 4, (chart_height * 6) + 22 + i, chemical_cols - 1],
            'line':       {'color': colours_hex[i], 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('B114', chem1)

    ############################################################################################################################

    # Create a pulp paper production line chart
    pulppaper1 = workbook.add_chart({'type': 'line'})
    pulppaper1.set_size({
        'width': 700,
        'height': 300
    })
    
    pulppaper1.set_chartarea({
        'border': {'none': True}
    })
    
    pulppaper1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    pulppaper1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Pulp paper production (tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    pulppaper1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10}
        #'none': True
    })
        
    pulppaper1.set_title({
        'none': True
    })
        
    for item in pulp_paper:
        i = pulppaper_df1[pulppaper_df1['item'] == item].index[0]
        pulppaper1.add_series({
            'name':       [economy, (chart_height * 7) + 29 + i, 1],
            'categories': [economy, (chart_height * 7) + 28, 4, (chart_height * 7) + 28, pulppaper_cols - 1],
            'values':     [economy, (chart_height * 7) + 29 + i, 4, (chart_height * 7) + 29 + i, pulppaper_cols - 1],
            'line':       {'color': colours_hex[i], 'width': 1} 
        })
    
    econ_worksheet.insert_chart('B139', pulppaper1)

    #################################################################################################################################

    # Create an mining line chart
    mining1 = workbook.add_chart({'type': 'line'})
    mining1.set_size({
        'width': 500,
        'height': 300
    })
    
    mining1.set_chartarea({
        'border': {'none': True}
    })
    
    mining1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    mining1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Mining GVA',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    mining1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    mining1.set_title({
        'none': True
    })
        
    mining1.add_series({
        'name':       [economy, (chart_height * 8) + 38, 1],
        'categories': [economy, (chart_height * 8) + 37, 4, (chart_height * 8) + 37, mining_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 8) + 38, 4, (chart_height * 8) + 38, mining_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    
    econ_worksheet.insert_chart('B166', mining1)

    #################################################################################################################################

    # Create an food and bev gva line chart
    foodbev1 = workbook.add_chart({'type': 'line'})
    foodbev1.set_size({
        'width': 500,
        'height': 300
    })
    
    foodbev1.set_chartarea({
        'border': {'none': True}
    })
    
    foodbev1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    foodbev1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Food and beverage GVA',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    foodbev1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    foodbev1.set_title({
        'none': True
    })
        
    foodbev1.add_series({
        'name':       [economy, (chart_height * 9) + 40, 1],
        'categories': [economy, (chart_height * 9) + 39, 4, (chart_height * 9) + 39, fb_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 9) + 40, 4, (chart_height * 9) + 40, fb_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B186', foodbev1)

    #################################################################################################################################

    # Create an textiles gva line chart
    textiles1 = workbook.add_chart({'type': 'line'})
    textiles1.set_size({
        'width': 500,
        'height': 300
    })
    
    textiles1.set_chartarea({
        'border': {'none': True}
    })
    
    textiles1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    textiles1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Textiles GVA',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    textiles1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    textiles1.set_title({
        'none': True
    })
        
    textiles1.add_series({
        'name':       [economy, (chart_height * 10) + 42, 1],
        'categories': [economy, (chart_height * 10) + 41, 4, (chart_height * 10) + 41, textiles_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 10) + 42, 4, (chart_height * 10) + 42, textiles_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B206', textiles1)

    #################################################################################################################################

    # Create an wood and wood products gva line chart
    wood1 = workbook.add_chart({'type': 'line'})
    wood1.set_size({
        'width': 500,
        'height': 300
    })
    
    wood1.set_chartarea({
        'border': {'none': True}
    })
    
    wood1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    wood1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Wood and wood products GVA',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    wood1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    wood1.set_title({
        'none': True
    })
        
    wood1.add_series({
        'name':       [economy, (chart_height * 11) + 44, 1],
        'categories': [economy, (chart_height * 11) + 43, 4, (chart_height * 11) + 43, wood_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 11) + 44, 4, (chart_height * 11) + 44, wood_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B226', wood1)

    #################################################################################################################################

    # Create an machinery line chart
    machinery1 = workbook.add_chart({'type': 'line'})
    machinery1.set_size({
        'width': 500,
        'height': 300
    })
    
    machinery1.set_chartarea({
        'border': {'none': True}
    })
    
    machinery1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    machinery1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Machinery GVA',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    machinery1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    machinery1.set_title({
        'none': True
    })
        
    machinery1.add_series({
        'name':       [economy, (chart_height * 12) + 46, 1],
        'categories': [economy, (chart_height * 12) + 45, 4, (chart_height * 12) + 45, machinery_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 12) + 46, 4, (chart_height * 12) + 46, machinery_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B246', machinery1)

    #################################################################################################################################

    # Create an vehicles gva line chart
    vehicles1 = workbook.add_chart({'type': 'line'})
    vehicles1.set_size({
        'width': 500,
        'height': 300
    })
    
    vehicles1.set_chartarea({
        'border': {'none': True}
    })
    
    vehicles1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    vehicles1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Vehicles GVA',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    vehicles1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    vehicles1.set_title({
        'none': True
    })
        
    vehicles1.add_series({
        'name':       [economy, (chart_height * 13) + 48, 1],
        'categories': [economy, (chart_height * 13) + 47, 4, (chart_height * 13) + 47, vehicles_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 13) + 48, 4, (chart_height * 13) + 48, vehicles_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B266', vehicles1)

    #################################################################################################################################

    # Create an other transport gva line chart
    oth_trans1 = workbook.add_chart({'type': 'line'})
    oth_trans1.set_size({
        'width': 500,
        'height': 300
    })
    
    oth_trans1.set_chartarea({
        'border': {'none': True}
    })
    
    oth_trans1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    oth_trans1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Other transport GVA',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    oth_trans1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    oth_trans1.set_title({
        'none': True
    })
        
    oth_trans1.add_series({
        'name':       [economy, (chart_height * 14) + 50, 1],
        'categories': [economy, (chart_height * 14) + 49, 4, (chart_height * 14) + 49, oth_trans_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 14) + 50, 4, (chart_height * 14) + 50, oth_trans_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B286', oth_trans1)

    #################################################################################################################################

    # Create an other transport gva line chart
    construction1 = workbook.add_chart({'type': 'line'})
    construction1.set_size({
        'width': 500,
        'height': 300
    })
    
    construction1.set_chartarea({
        'border': {'none': True}
    })
    
    construction1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    construction1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Construction GVA',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    construction1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    construction1.set_title({
        'none': True
    })
        
    construction1.add_series({
        'name':       [economy, (chart_height * 15) + 52, 1],
        'categories': [economy, (chart_height * 15) + 51, 4, (chart_height * 15) + 51, construction_df1.shape[1] - 1],
        'values':     [economy, (chart_height * 15) + 52, 4, (chart_height * 15) + 52, construction_df1.shape[1] - 1],
        'line':       {'color': 'blue', 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B306', construction1)

writer.save()